# ALS basic

Idea: Assume the (true) matrix X can be decomposed into two low rank matrices U and V s.t X = UV. Approximate this matrix factorization by minimizing the regularized Frobenius loss. 

$L(U,V) = \sum_{(i,j)\in I} (a_{ij}-u_i^Tv_j)^2 + \lambda \sum_{i=1}^m \lVert u_i \rVert^2 + \lambda \sum_{j=1}^n \lVert v_j \rVert^2 $

Alternating Least Squares minimizes $L(U,V)$ by alternatly updating U and V based on the gradient of L.